# Sentiment Analysis with Transfer Learning and Fine-tuning

This notebook demonstrates how to fine-tune a pre-trained model (DistilBERT) for a binary sentiment analysis task.

## 1. Setup and Imports

In [1]:
%pip install torch transformers pandas scikit-learn datasets


     ---------------------------------------- 0.0/199.4 MB ? eta -:--:--
     ---------------------------------------- 0.2/199.4 MB 4.6 MB/s eta 0:00:44
     ---------------------------------------- 0.5/199.4 MB 5.9 MB/s eta 0:00:34
     ---------------------------------------- 1.0/199.4 MB 7.7 MB/s eta 0:00:26
     ---------------------------------------- 1.5/199.4 MB 8.6 MB/s eta 0:00:23
     ---------------------------------------- 2.0/199.4 MB 9.2 MB/s eta 0:00:22
      --------------------------------------- 2.5/199.4 MB 9.5 MB/s eta 0:00:21
      --------------------------------------- 3.1/199.4 MB 9.8 MB/s eta 0:00:21
      -------------------------------------- 3.6/199.4 MB 10.0 MB/s eta 0:00:20
      -------------------------------------- 4.1/199.4 MB 10.1 MB/s eta 0:00:20
      -------------------------------------- 4.6/199.4 MB 10.2 MB/s eta 0:00:20
     - ------------------------------------- 5.2/199.4 MB 10.3 MB/s eta 0:00:19
     - ------------------------------------- 5.


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import numpy as np

print(f"PyTorch version: {torch.__version__}")
print(f"GPU available: {torch.cuda.is_available()}")

PyTorch version: 2.4.1+cpu
GPU available: False


## 2. Data Loading and Preparation

In [3]:
from datasets import load_dataset
import pandas as pd
import os

In [4]:
dataset = load_dataset("imdb")

In [ ]:
print(dataset)

In [7]:
def load_imdb_dataset(max_samples=10000):
    df = pd.read_csv('IMDB_dataset.csv')  # Make sure you have this file
    df = df.sample(n=min(max_samples, len(df)), random_state=42)
    texts = df['review'].tolist()
    labels = (df['sentiment'] == 'positive').astype(int).tolist()
    return train_test_split(texts, labels, test_size=0.2, random_state=42)

train_texts, val_texts, train_labels, val_labels = load_imdb_dataset()

print(f"Training set size: {len(train_texts)}")
print(f"Validation set size: {len(val_texts)}")

FileNotFoundError: [Errno 2] No such file or directory: 'IMDB Dataset.csv'

## 3. Custom Dataset Creation

In [ ]:
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

## 4. Model and Tokenizer Initialization

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

max_length = 128  # Adjust based on your needs and computational constraints
train_dataset = SentimentDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = SentimentDataset(val_texts, val_labels, tokenizer, max_length)

batch_size = 16  # Adjust based on your GPU memory
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

print(f"Number of training batches: {len(train_dataloader)}")
print(f"Number of validation batches: {len(val_dataloader)}")

## 5. Optimizer Configuration

In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(f"Using device: {device}")

## 6. Training and Evaluation Functions

In [ ]:
def train_epoch(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in dataloader:
        optimizer.zero_grad()
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**inputs)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    return total_loss / len(dataloader)

def evaluate(model, dataloader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in dataloader:
            inputs = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**inputs)
            preds = torch.argmax(outputs.logits, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(inputs['labels'].cpu().tolist())
    
    accuracy = accuracy_score(actual_labels, predictions)
    f1 = f1_score(actual_labels, predictions, average='binary')
    return accuracy, f1

## 7. Main Training Loop

In [ ]:
num_epochs = 3  # Adjust based on your needs and time constraints

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    avg_train_loss = train_epoch(model, train_dataloader, optimizer, device)
    print(f"Average training loss: {avg_train_loss:.4f}")
    
    accuracy, f1 = evaluate(model, val_dataloader, device)
    print(f"Validation Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}")
    print("----")

print("Training completed!")

## 8. Save the Model

In [ ]:
model.save_pretrained('./models/sentiment_model')
tokenizer.save_pretrained('./models/sentiment_model')
print("Model saved in './modles/sentiment_model' directory")

## 9. Test the Model

In [ ]:
def predict_sentiment(text, model, tokenizer, device):
    model.eval()
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        prediction = torch.argmax(outputs.logits, dim=1)
    
    return "Positive" if prediction.item() == 1 else "Negative"

# Test the model with some example reviews
test_reviews = [
    "This movie was fantastic! I loved every minute of it.",
    "Absolutely terrible. Waste of time and money.",
    "It was okay, nothing special but not bad either."
]

for review in test_reviews:
    sentiment = predict_sentiment(review, model, tokenizer, device)
    print(f"Review: {review}")
    print(f"Predicted sentiment: {sentiment}")
    print("----")